Le dataframe utilisé pour Ce test est le df hospital 20k netoyé il contient sept contraintes valides :

```
"MeasureCode,State → StateAvg",
"ZIPCode → State",
"PhoneNumber → ZIPCode",
"MeasureCode → MeasureName",
"MeasureCode → Condition",
"PhoneNumber → City",
"PhoneNumber → State"
```




Le but est de soumettre un ensemble de contraintes erronées en entrée, en supposant que l'utilisateur a commis des erreurs en manipulant des attributs, en ajoutant des attributs supplémentaires ou en supprimant des attributs. Ensuite, nous vérifions si le programme peut les réparer pour obtenir des contraintes valides. Dans le cas où cela n'est pas possible, une réparation hybride est effectuée en adoptant la contrainte qui respecte le plus grand nombre de lignes et en adaptant les données à celle-ci. À la fin, les contraintes du système seront modifiées en remplaçant les contraintes erronées par leurs réparations.






# **1.   Implémentation**




**Installation :**

In [1]:
!pip install psutil


**Importations**

In [2]:
import pandas as pd
from sklearn.metrics import mutual_info_score
import numpy as np
from sklearn.preprocessing import LabelEncoder
import tensorflow_probability as tfp
import scipy
from sklearn.metrics import mutual_info_score
import itertools
from sklearn.metrics import normalized_mutual_info_score
import time
import psutil



**Connexion drive**

In [3]:
# pour monter son drive Google Drive local
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import sys
my_local_drive='/content/drive/MyDrive/PFE'

# Ajout du path pour les librairies, fonctions et données
sys.path.append(my_local_drive)
# Se positionner sur le répertoire associé
%cd $my_local_drive

%pwd


/content/drive/MyDrive/PFE


'/content/drive/MyDrive/PFE'

**Chargement des donnes**

In [5]:


df = pd.read_csv('/content/drive/MyDrive/PFE/hospital_20k_Clean.csv')
print(df)
df = df.drop(columns=['row_num'])

print(df)


       row_num  ProviderNumber        City State  ZIPCode  PhoneNumber  \
0            1           10018  BIRMINGHAM    AL    35233   2053258100   
1            2           10018  BIRMINGHAM    AL    35233   2053258100   
2            3           10018  BIRMINGHAM    AL    35233   2053258100   
3            4           10018  BIRMINGHAM    AL    35233   2053258100   
4            5           10018  BIRMINGHAM    AL    35233   2053258100   
...        ...             ...         ...   ...      ...          ...   
19995    19996          100118  PALM COAST    FL    32164   3865862000   
19996    19997          100118  PALM COAST    FL    32164   3865862000   
19997    19998          100118  PALM COAST    FL    32164   3865862000   
19998    19999          100118  PALM COAST    FL    32164   3865862000   
19999    20000          100118  PALM COAST    FL    32164   3865862000   

                           Condition  MeasureCode  \
0      Surgical Infection Prevention  SCIP-CARD-2   
1    

**encodage des données**

In [6]:

# Encoder les attributs catégoriels en valeurs numériques
label_encoder = LabelEncoder()
df_encoded = df.copy()
for col in df_encoded.columns:
    if df_encoded[col].dtype == 'object':
        df_encoded[col] = label_encoder.fit_transform(df_encoded[col])

df=df_encoded
print(df)


       ProviderNumber  City  State  ZIPCode  PhoneNumber  Condition  \
0               10018    38      1    35233   2053258100          4   
1               10018    38      1    35233   2053258100          4   
2               10018    38      1    35233   2053258100          4   
3               10018    38      1    35233   2053258100          4   
4               10018    38      1    35233   2053258100          4   
...               ...   ...    ...      ...          ...        ...   
19995          100118   360      9    32164   3865862000          1   
19996          100118   360      9    32164   3865862000          1   
19997          100118   360      9    32164   3865862000          1   
19998          100118   360      9    32164   3865862000          1   
19999          100118   360      9    32164   3865862000          2   

       MeasureCode  MeasureName  StateAvg  
0               20           25        48  
1               21           23        49  
2              

**Ajouter des attributs à gauche**

In [7]:


def generer_combinaisons(attributs_sup_jeme_moy, j):
    # Utiliser itertools.combinations pour générer les combinaisons de taille j
    combinaisons = list(itertools.combinations(attributs_sup_jeme_moy, j))

    return combinaisons

def ajouter_attributs_gauche(df, contrainte_erronee, n):
    # Extraire les attributs du côté gauche et du côté droit de la flèche "→"
    attributs_gauche, attributs_droite = [part.strip() for part in contrainte_erronee.split(' → ')]

    # Convertir les listes en ensembles pour faciliter les opérations ensemblistes
    attributs_droite = set(attributs_droite.split(', '))
    attributs_gauche = set(attributs_gauche.split(', '))

    # Calculer les attributs restants (tous les attributs - attributs du côté droit - attributs du côté gauche)
    attributs_tous = set(df.columns)
    attributs_restants = attributs_tous - attributs_droite - attributs_gauche

    if n is None:
        n = 2

    # Étape 1 : Initialiser les listes pour stocker les informations mutuelles et les moyennes
    info_mutuelles = []
    moyennes_info_mutuelles = []

    # Étape 2 : Calculer l'information mutuelle pour chaque paire d'éléments
    for attr_r in attributs_restants:
        info_mutuelle_attr_r = []
        for attr_d in attributs_droite:
            # Récupérer les données correspondant aux attributs d'intérêt
            data_d = df[attr_d]
            data_r = df[attr_r]

            # Calculer l'information mutuelle entre les deux attributs
            info_mutuelle = normalized_mutual_info_score(data_d, data_r)

            info_mutuelle_attr_r.append(info_mutuelle)

        info_mutuelles.append(info_mutuelle_attr_r)

    # Étape 3 : Calculer les moyennes d'information mutuelle pour chaque attribut restant
    for info_mutuelle_attr_r in info_mutuelles:
        moyenne_info_mutuelle = np.mean(info_mutuelle_attr_r)

        moyennes_info_mutuelles.append(moyenne_info_mutuelle)

    # Étape 4 : Trier les attributs restants en fonction des moyennes d'information mutuelle calculées
    resultats_tries = [x for _, x in sorted(zip(moyennes_info_mutuelles, attributs_restants), reverse=True)]

    moyennes_info_mutuelles.sort(reverse=True)

    # Trouver l'attribut avec la moyenne la plus élevée parmi tous les attributs triés
    attribut_max_moyenne = [attr for attr in resultats_tries if moyennes_info_mutuelles[resultats_tries.index(attr)] == moyennes_info_mutuelles[0]]
    # Convertir l'ensemble attributs_gauche en liste
    attributs_gauche_liste = list(attributs_gauche)

    # Étape 5 : Construire les nouvelles contraintes avec l'attribut ayant la meilleure moyenne d'information mutuelle
    nouvelles_contraintes = []
    for attr in attribut_max_moyenne:
        nouvelles_contrainte = f"{', '.join(attributs_gauche_liste + [attr])} → {', '.join(attributs_droite)}"
        nouvelles_contraintes.append(nouvelles_contrainte)
     # Étape 6 : Trouver les  attributs avec les n meilleurs moyennes
    if n > 1:
        for j in range(2, n + 1):
            attributs_sup_jeme_moy = [attr for attr in resultats_tries if moyennes_info_mutuelles[resultats_tries.index(attr)] >= moyennes_info_mutuelles[j - 1]]
            combinaisons_attributs = generer_combinaisons(attributs_sup_jeme_moy, j)

            # Étape 7 : Ajouter les combinaisons au côté gauche de la contrainte pour créer de nouvelles contraintes
            for combinaison in combinaisons_attributs:
                nouvelle_contrainte = f"{', '.join(attributs_gauche_liste + list(combinaison))} → {', '.join(attributs_droite)}"
                nouvelles_contraintes.append(nouvelle_contrainte)

    return nouvelles_contraintes




In [8]:
# Exemple d'utilisation de la fonction ajouter_attributs_gauche()
contrainte_erronee = "State  → City"
nouvelles_contraintes = ajouter_attributs_gauche(df, contrainte_erronee, None)
for nouvelle_contrainte in nouvelles_contraintes:
    print(nouvelle_contrainte)

State, ZIPCode → City
State, ZIPCode, PhoneNumber → City


**Ajouter des attributs à droite**

In [9]:


def generer_combinaisons(attributs_sup_jeme_moy, j):
    # Utiliser itertools.combinations pour générer les combinaisons de taille j
    combinaisons = list(itertools.combinations(attributs_sup_jeme_moy, j))

    return combinaisons

def ajouter_attributs_droite(df, contrainte_erronee, n):
    # Extraire les attributs du côté gauche et du côté droit de la flèche "→"
    attributs_gauche, attributs_droite = [part.strip() for part in contrainte_erronee.split(' → ')]

    # Convertir les listes en ensembles pour faciliter les opérations ensemblistes
    attributs_droite = set(attributs_droite.split(', '))
    attributs_gauche = set(attributs_gauche.split(', '))

    # Calculer les attributs restants (tous les attributs - attributs du côté droit - attributs du côté gauche)
    attributs_tous = set(df.columns)
    attributs_restants = attributs_tous - attributs_droite - attributs_gauche

    if n is None:
        n = 2

    # Étape 1 : Initialiser les listes pour stocker les informations mutuelles et les moyennes
    info_mutuelles = []
    moyennes_info_mutuelles = []

    # Étape 2 : Calculer l'information mutuelle pour chaque paire d'éléments
    for attr_r in attributs_restants:
        info_mutuelle_attr_r = []
        for attr_d in attributs_gauche:
            # Récupérer les données correspondant aux attributs d'intérêt
            data_d = df[attr_d]
            data_r = df[attr_r]

            # Calculer l'information mutuelle entre les deux attributs
            info_mutuelle = mutual_info_score(data_d, data_r)

            info_mutuelle_attr_r.append(info_mutuelle)

        info_mutuelles.append(info_mutuelle_attr_r)

    # Étape 3 : Calculer les moyennes d'information mutuelle pour chaque attribut restant
    for info_mutuelle_attr_r in info_mutuelles:
        moyenne_info_mutuelle = np.mean(info_mutuelle_attr_r)

        moyennes_info_mutuelles.append(moyenne_info_mutuelle)

    # Étape 4 : Trier les attributs restants en fonction des moyennes d'information mutuelle calculées
    resultats_tries = [x for _, x in sorted(zip(moyennes_info_mutuelles, attributs_restants), reverse=True)]

    moyennes_info_mutuelles.sort(reverse=True)

    # Trouver l'attribut avec la moyenne la plus élevée parmi tous les attributs triés
    attribut_max_moyenne = [attr for attr in resultats_tries if moyennes_info_mutuelles[resultats_tries.index(attr)] == moyennes_info_mutuelles[0]]
    # Convertir l'ensemble attributs_droite en liste
    attributs_droite_liste = list(attributs_droite)

    # Étape 5 : Construire les nouvelles contraintes avec l'attribut ayant la meilleure moyenne d'information mutuelle
    nouvelles_contraintes = []
    for attr in attribut_max_moyenne:

        nouvelles_contrainte = f"{', '.join(attributs_gauche)} → {', '.join(attributs_droite_liste + [attr])}"
        nouvelles_contraintes.append(nouvelles_contrainte)
    # Étape 6 : Trouver les  attributs avec les n meilleurs moyennes
    if n > 1:
        for j in range(2, n + 1):
            attributs_sup_jeme_moy = [attr for attr in resultats_tries if moyennes_info_mutuelles[resultats_tries.index(attr)] >= moyennes_info_mutuelles[j - 1]]

            combinaisons_attributs = generer_combinaisons(attributs_sup_jeme_moy, j)

            # Étape 7 : Ajouter les combinaisons au côté droite de la contrainte pour créer de nouvelles contraintes
            for combinaison in combinaisons_attributs:

                nouvelle_contrainte = f"{', '.join(attributs_gauche)} → {', '.join(attributs_droite_liste + list(combinaison))}"
                nouvelles_contraintes.append(nouvelle_contrainte)

    return nouvelles_contraintes



In [10]:
# Exemple d'utilisation de la fonction ajouter_attributs_droite()
contrainte_erronee = "State  → City, PhoneNumber"
nouvelles_contraintes = ajouter_attributs_droite(df, contrainte_erronee, None)
for nouvelle_contrainte in nouvelles_contraintes:
    print(nouvelle_contrainte)

State → City, PhoneNumber, ZIPCode
State → City, PhoneNumber, ZIPCode, StateAvg


**supression des attributs à droite**

In [11]:


def generer_combinaisons(attributs_sup_jeme_moy, j):
    # Utiliser itertools.combinations pour générer les combinaisons de taille j
    combinaisons = list(itertools.combinations(attributs_sup_jeme_moy, j))

    return combinaisons

def supprimer_attributs_droite(df, contrainte_erronee, n):
    # Extraire les attributs du côté gauche et du côté droit de la flèche "→"
    attributs_gauche, attributs_droite = [part.strip() for part in contrainte_erronee.split(' → ')]

    # Convertir les listes en ensembles pour faciliter les opérations ensemblistes
    attributs_droite = set(attributs_droite.split(', '))
    attributs_gauche = set(attributs_gauche.split(', '))

    if n is None:
        n = len(attributs_droite)
    if n == 0:
            return None
    else:
            # Étape 1 : Initialiser les listes pour stocker les informations mutuelles et les moyennes
            info_mutuelles = []
            moyennes_info_mutuelles = []

            # Étape 2 : Calculer l'information mutuelle pour chaque paire d'éléments
            for attr_r in attributs_droite:
                info_mutuelle_attr_r = []
                for attr_d in attributs_gauche:
                    # Récupérer les données correspondant aux attributs d'intérêt
                    data_d = df[attr_d]
                    data_r = df[attr_r]

                    # Calculer l'information mutuelle entre les deux attributs
                    info_mutuelle = mutual_info_score(data_d, data_r)

                    info_mutuelle_attr_r.append(info_mutuelle)

                info_mutuelles.append(info_mutuelle_attr_r)

            # Étape 3 : Calculer les moyennes d'information mutuelle pour chaque attribut droite
            for info_mutuelle_attr_r in info_mutuelles:
                moyenne_info_mutuelle = np.mean(info_mutuelle_attr_r)

                moyennes_info_mutuelles.append(moyenne_info_mutuelle)

            # Étape 4 : Trier les attributs droites en fonction des moyennes d'information mutuelle calculées
            resultats_tries = [x for _, x in sorted(zip(moyennes_info_mutuelles, attributs_droite), reverse=False)]

            moyennes_info_mutuelles.sort(reverse=False)

            # Trouver l'attribut avec la moyenne la plus basse parmi tous les attributs triés
            attribut_min_moyenne = [attr for attr in resultats_tries if moyennes_info_mutuelles[resultats_tries.index(attr)] == moyennes_info_mutuelles[0]]

            # Convertir l'ensemble attributs_droite en liste
            attributs_droite_liste = list(attributs_droite)

            # Étape 5 : Construire les nouvelles contraintes en enlevant l'attribut ayant la  moyenne la plus basse  d'information mutuelle
            nouvelles_contraintes = []
            for attr in attribut_min_moyenne:
                # Créez une copie de la liste attributs_droite_liste
                attributs_droite_liste_copy = attributs_droite_liste.copy()

                # Supprimez l'attribut de la copie
                attributs_droite_liste_copy.remove(attr)

                # Créez la nouvelle contrainte avec la copie modifiée
                nouvelles_contrainte = f"{', '.join(attributs_gauche)} → {', '.join(attributs_droite_liste_copy)}"

                # Ajoutez la nouvelle contrainte à la liste nouvelles_contraintes
                nouvelles_contraintes.append(nouvelles_contrainte)
            # Étape 6 : Trouver les  attributs avec les n mauvaises moyennes
            if n > 1:
                for j in range(2, n + 1):
                    attributs_sup_jeme_moy = [attr for attr in resultats_tries if moyennes_info_mutuelles[resultats_tries.index(attr)] <= moyennes_info_mutuelles[j - 1]]

                    combinaisons_attributs = generer_combinaisons(attributs_sup_jeme_moy, j)

                    # Étape 7 : Enlever les combinaisons du côté droite de la contrainte pour créer de nouvelles contraintes
                    for combinaison in combinaisons_attributs:
                        nouvelle_contrainte = f"{', '.join(attributs_gauche)} → {', '.join([attr for attr in attributs_droite_liste if attr not in combinaison])}"

                        nouvelles_contraintes.append(nouvelle_contrainte)

            return nouvelles_contraintes




In [12]:
# Exemple d'utilisation de la fonction supprimer_attributs_droite
contrainte_erronee = "PhoneNumber → State, City"
nouvelles_contraintes = supprimer_attributs_droite(df, contrainte_erronee, None)
for nouvelle_contrainte in nouvelles_contraintes:
    print(nouvelle_contrainte)

PhoneNumber → City
PhoneNumber → 


**supression des attribut à gauche**

In [13]:


def generer_combinaisons(attributs_sup_jeme_moy, j):
    # Utiliser itertools.combinations pour générer les combinaisons de taille j
    combinaisons = list(itertools.combinations(attributs_sup_jeme_moy, j))

    return combinaisons

def supprimer_attributs_gauche(df, contrainte_erronee, n):
    # Extraire les attributs du côté gauche et du côté droit de la flèche "→"
    attributs_gauche, attributs_droite = [part.strip() for part in contrainte_erronee.split(' → ')]

    # Convertir les listes en ensembles pour faciliter les opérations ensemblistes
    attributs_droite = set(attributs_droite.split(', '))
    attributs_gauche = set(attributs_gauche.split(', '))



    if n is None:
        n = len(attributs_gauche)
    if n == 0:
            return None
    else:

            # Étape 1 : Initialiser les listes pour stocker les informations mutuelles et les moyennes
            info_mutuelles = []
            moyennes_info_mutuelles = []

            # Étape 2 : Calculer l'information mutuelle pour chaque paire d'éléments
            for attr_r in attributs_gauche:
                info_mutuelle_attr_r = []
                for attr_d in attributs_droite:
                    # Récupérer les données correspondant aux attributs d'intérêt
                    data_d = df[attr_d]
                    data_r = df[attr_r]

                    # Calculer l'information mutuelle entre les deux attributs
                    info_mutuelle = mutual_info_score(data_d, data_r)

                    info_mutuelle_attr_r.append(info_mutuelle)

                info_mutuelles.append(info_mutuelle_attr_r)

            # Étape 3 : Calculer les moyennes d'information mutuelle pour chaque attribut gauche
            for info_mutuelle_attr_r in info_mutuelles:
                moyenne_info_mutuelle = np.mean(info_mutuelle_attr_r)

                moyennes_info_mutuelles.append(moyenne_info_mutuelle)

            # Étape 4 : Trier les attributs gauches en fonction des moyennes d'information mutuelle calculées
            resultats_tries = [x for _, x in sorted(zip(moyennes_info_mutuelles, attributs_gauche), reverse=False)]

            moyennes_info_mutuelles.sort(reverse=False)

            # Trouver l'attribut avec la moyenne la plus basses parmi tous les attributs triés
            attribut_min_moyenne = [attr for attr in resultats_tries if moyennes_info_mutuelles[resultats_tries.index(attr)] == moyennes_info_mutuelles[0]]

            # Convertir l'ensemble attributs_gauche en liste
            attributs_gauche_liste = list(attributs_gauche)

            # Étape 5 : Construire les nouvelles contraintes avec l'attribut ayant la mauvaise moyenne d'information mutuelle
            nouvelles_contraintes = []
            for attr in attribut_min_moyenne:
              # Créez une copie de la liste attributs_gauche_liste
              attributs_gauche_liste_copy = attributs_gauche_liste.copy()

              # Supprimez l'attribut de la copie
              attributs_gauche_liste_copy.remove(attr)

              # Créez la nouvelle contrainte avec la copie modifiée
              nouvelles_contrainte = f"{', '.join(attributs_gauche_liste_copy)} → {', '.join(attributs_droite)}"

              # Ajoutez la nouvelle contrainte à la liste nouvelles_contraintes
              nouvelles_contraintes.append(nouvelles_contrainte)

            # Étape 6 : Trouver les  attributs avec les n mauvaises moyennes
            if n > 1:
                for j in range(2, n + 1):
                    attributs_sup_jeme_moy = [attr for attr in resultats_tries if moyennes_info_mutuelles[resultats_tries.index(attr)] <= moyennes_info_mutuelles[j-1]]

                    combinaisons_attributs = generer_combinaisons(attributs_sup_jeme_moy, j)

                    # Étape 7 : supprimer les combinaisons du côté gauche de la contrainte pour créer de nouvelles contraintes
                    for combinaison in combinaisons_attributs:

                        nouvelle_contrainte = f"{', '.join  ([attr for attr in attributs_gauche_liste if attr not in combinaison])} → {', '.join(attributs_droite)}"


                        nouvelles_contraintes.append(nouvelle_contrainte)

            return nouvelles_contraintes




In [14]:
# Exemple d'utilisation de la fonction trier_attributs_par_info_mutuelle
contrainte_erronee = "State  → City"
nouvelles_contraintes = supprimer_attributs_gauche(df, contrainte_erronee, None)
for nouvelle_contrainte in nouvelles_contraintes:
    print(nouvelle_contrainte)

 → City



**Remplacemnt à gauche**

In [15]:
def remplacer_attribut_gauche(df, contrainte_errone):


    nouvelles_contraintes_supprimees = supprimer_attributs_gauche(df, contrainte_errone, 1)
    nouvelles_contraintes = []

    for nouvelle_contrainte_supprimee in nouvelles_contraintes_supprimees:
        nouvelles_contraintes_ajoutees = ajouter_attributs_gauche(df, nouvelle_contrainte_supprimee, 1)
        nouvelles_contraintes.extend(nouvelles_contraintes_ajoutees)

    return nouvelles_contraintes


In [16]:
# Exemple d'utilisation de la fonction remplacer_attribut_gauche
contrainte_erronee = "MeasureCode, State → StateAvg"
nouvelles_contraintes = remplacer_attribut_gauche(df, contrainte_erronee)
for nouvelle_contrainte in nouvelles_contraintes:
    print(nouvelle_contrainte)


MeasureCode, MeasureName → StateAvg


**Remplacement à droite**

In [17]:
def remplacer_attribut_droite(df, contrainte_errone):


    nouvelles_contraintes_supprimees = supprimer_attributs_droite(df, contrainte_errone, 1)
    nouvelles_contraintes = []

    for nouvelle_contrainte_supprimee in nouvelles_contraintes_supprimees:
        nouvelles_contraintes_ajoutees = ajouter_attributs_droite(df, nouvelle_contrainte_supprimee, 1)
        nouvelles_contraintes.extend(nouvelles_contraintes_ajoutees)

    return nouvelles_contraintes



In [18]:
# Exemple d'utilisation de la fonction remplacer_attribut_droite
contrainte_erronee = "ZIPCode → City"
nouvelles_contraintes = remplacer_attribut_droite(df, contrainte_erronee)
for nouvelle_contrainte in nouvelles_contraintes:
    print(nouvelle_contrainte)

ZIPCode → , PhoneNumber


**Génération de toutes les Modifications de la contrainte erronee en utilisant les fonctions d'ajout, de supressuin et de remplacement déja écrites**

In [19]:
def GenererModificationsContraintes(dataframe, contrainte_erronee):
    contraintes_generees = []  # liste vide pour stocker les contraintes générées

    ajouter_attributs_gauche_resultat = ajouter_attributs_gauche(df, contrainte_erronee, None)
    contraintes_generees.extend(ajouter_attributs_gauche_resultat)

    ajouter_attributs_droite_resultat = ajouter_attributs_droite(df, contrainte_erronee,None)
    contraintes_generees.extend(ajouter_attributs_droite_resultat)

    supprimer_attributs_droite_resultat = supprimer_attributs_droite(df, contrainte_erronee, None)
    contraintes_generees.extend(supprimer_attributs_droite_resultat)

    supprimer_attributs_gauche_resultat = supprimer_attributs_gauche(df, contrainte_erronee, None)
    contraintes_generees.extend(supprimer_attributs_gauche_resultat)

    remplacer_attribut_gauche_resultat = remplacer_attribut_gauche(df, contrainte_erronee)
    contraintes_generees.extend(remplacer_attribut_gauche_resultat)

    remplacer_attribut_droite_resultat = remplacer_attribut_droite(df, contrainte_erronee)
    contraintes_generees.extend(remplacer_attribut_droite_resultat)

    return contraintes_generees


In [20]:
# Exemple d'utilisation de la fonction GenererModificationsContraintes
contrainte_erronee = "PhoneNumber → State, City"

contraintes_generees = GenererModificationsContraintes(df, contrainte_erronee)
for nouvelle_contrainte in contraintes_generees:
    print(nouvelle_contrainte)

PhoneNumber, ZIPCode → City, State
PhoneNumber, ZIPCode, ProviderNumber → City, State
PhoneNumber → City, State, ProviderNumber
PhoneNumber → City, State, ProviderNumber, ZIPCode
PhoneNumber → City
PhoneNumber → 
 → City, State
, ZIPCode → City, State
PhoneNumber → City, ProviderNumber


**Nettoyage des contrainte générées(mettre au bon format)**

In [21]:
def filtrer_contraintes(contraintes_generees):    # fonction pour supprime les contraintes qu ont pas l'un des 2 cotées
    contraintes_filtrees = []
    for nouvelle_contrainte in contraintes_generees:
        # Séparer les deux côtés de la contrainte
        cote_gauche, cote_droit = nouvelle_contrainte.split(" → ")

        # Vérifier si les deux côtés existent
        if cote_gauche and cote_droit:
            contraintes_filtrees.append(nouvelle_contrainte)

    return contraintes_filtrees


def nettoyer_contrainte(contrainte):     # fonction pou Supprimer les virgules inutiles
    cote_gauche, cote_droit = contrainte.split(" → ")

    # Supprimer la virgule du côté gauche s'il commence par une virgule
    if cote_gauche.startswith(","):
        cote_gauche = cote_gauche[1:].strip()

    # Supprimer la virgule du côté droit s'il commence par une virgule
    if cote_droit.startswith(","):
        cote_droit = cote_droit[1:].strip()

    return f"{cote_gauche} → {cote_droit}"

def nettoyer_contraintes(contraintes_generees):
    contraintes_nettoyees = []
    for nouvelle_contrainte in contraintes_generees:
        contrainte_modifiee = nettoyer_contrainte(nouvelle_contrainte)
        contraintes_nettoyees.append(contrainte_modifiee)

    return contraintes_nettoyees



In [22]:
# Exemple d'utilisation des fonction nettoyer_contraintes et filtrer_contraintes
contrainte_erronee = "PhoneNumber → State, Condition"
contraintes_generees = GenererModificationsContraintes(df, contrainte_erronee)

# Filtrer et nettoyer les contraintes générées
contraintes_nettoyees = nettoyer_contraintes(contraintes_generees)
contraintes_filtrees = filtrer_contraintes(contraintes_nettoyees)

# Afficher les contraintes filtrées
for nouvelle_contrainte in contraintes_filtrees:
    print(nouvelle_contrainte)

PhoneNumber, StateAvg → Condition, State
PhoneNumber, StateAvg, MeasureName → Condition, State
PhoneNumber, StateAvg, MeasureCode → Condition, State
PhoneNumber, MeasureName, MeasureCode → Condition, State
PhoneNumber → Condition, State, ProviderNumber
PhoneNumber → Condition, State, ProviderNumber, ZIPCode
PhoneNumber → State
StateAvg → Condition, State
PhoneNumber → State, ProviderNumber


**Test de doublons (supression des contraintes générées existantes dans lesystème)**

In [23]:
def parse_fd(fd_str):
    # Diviser la chaîne de contrainte en parties gauche et droite
    left_side, right_side = fd_str.split(" → ")
    left_attributes = [attr.strip() for attr in left_side.split(",")]
    right_attributes = [attr.strip() for attr in right_side.split(",")]
    return (left_attributes, right_attributes)



def are_similar_constraint(fd_str1, fd_str2):
    fd1 = parse_fd(fd_str1)
    fd2 = parse_fd(fd_str2)

    # Convertir les contraintes en ensembles pour ignorer l'ordre des attributs
    left_side_fd1 = set(fd1[0])
    right_side_fd1 = set(fd1[1])

    left_side_fd2 = set(fd2[0])
    right_side_fd2 = set(fd2[1])

    # Vérifier si les côtés gauches et droits sont similaires
    left_similar = left_side_fd1 == left_side_fd2
    right_similar = right_side_fd1 == right_side_fd2


    return left_similar and right_similar







# Supposons que les contraintes du système actuel soient dans une liste comme celle-ci :
contraintes_systeme = ["MeasureCode, State → StateAvg", "ZIPCode → State", "PhoneNumber → ZIPCode", "ZIPCode → City", "MeasureCode → MeasureName", "MeasureCode → Condition", "PhoneNumber → City", "PhoneNumber → State"]

# Exemple d'utilisation :
contrainte_erronee = "State  → City"
contraintes_generees = GenererModificationsContraintes(df, contrainte_erronee)
# Filtrer et nettoyer les contraintes générées
contraintes_nettoyees = nettoyer_contraintes(contraintes_generees)
contraintes_filtrees = filtrer_contraintes(contraintes_nettoyees)

# Afficher les contraintes filtrées qui ne sont pas présentes dans le système
for nouvelle_contrainte in contraintes_filtrees:
    similar_present = any(are_similar_constraint(nouvelle_contrainte, contrainte_systeme) for contrainte_systeme in contraintes_systeme)
    if not similar_present:
        contraintes_systeme.append(nouvelle_contrainte)
        print(nouvelle_contrainte)

State, ZIPCode → City
State, ZIPCode, PhoneNumber → City
State → City, ZIPCode
State → City, PhoneNumber
State → City, ZIPCode, PhoneNumber
State → ZIPCode
State → PhoneNumber


**fonction pricipale pour generer les contraintes, les nettoyer , les filtrer et supprimer les contrainte du systeme**

In [24]:
def generer_contrainte(df, contrainte_erronee, contraintes_systemes):
    contraintes_testees = []

    contraintes_generees = GenererModificationsContraintes(df, contrainte_erronee)
    contraintes_nettoyees = nettoyer_contraintes(contraintes_generees)
    contraintes_filtrees = filtrer_contraintes(contraintes_nettoyees)
    for nouvelle_contrainte in contraintes_filtrees:
        similar_present = any(are_similar_constraint(nouvelle_contrainte, contrainte_systeme) for contrainte_systeme in contraintes_systemes)
        if not similar_present:
            contraintes_testees.append(nouvelle_contrainte)
    return contraintes_testees




In [25]:
# Exemple d'utilisation :
contrainte_erronee = "MeasureName → StateAvg"

# Supposons que les contraintes du système actuel soient dans une liste comme celle-ci :
contraintes_systemes = ["PhoneNumber → State, Condition"]

contraintes_testees = generer_contrainte(df, contrainte_erronee, contraintes_systemes)

# Afficher les contraintes filtrées
for nouvelle_contrainte in contraintes_testees:
    print(nouvelle_contrainte)

MeasureName, MeasureCode → StateAvg
MeasureName, MeasureCode, State → StateAvg
MeasureName → StateAvg, MeasureCode
MeasureName → StateAvg, MeasureCode, Condition
MeasureName → StateAvg
MeasureCode → StateAvg
MeasureName → MeasureCode


**Test de la validité des contraintes générées**

In [26]:


def check_functional_dependencies(df, constraints):
    valid_constraints = []

    for constraint in constraints:
        lhs, rhs = constraint.split(" → ")
        lhs = lhs.strip().split(', ')
        rhs = rhs.strip().split(', ')

        # Vérification si toutes les colonnes existent dans le DataFrame
        for col in lhs + rhs:
            if col not in df.columns:
                raise ValueError(f"La colonne '{col}' n'existe pas dans le DataFrame.")

        # Agréger les données en fonction de lhs et vérifier la dépendance fonctionnelle pour chaque combinaison unique
        grouped_data = df.groupby(lhs)[rhs].nunique()

        # Si le groupe unique a une seule valeur distincte pour chaque colonne rhs,
        # la contrainte est satisfaite pour toutes les lignes du DataFrame.
        if (grouped_data == 1).all().all():  # Ajouter un second .all() ici pour vérifier tous les groupes uniques.
            valid_constraints.append(constraint)

    return valid_constraints


In [27]:
# Example d'utulisation:

#contrainte_erronee = ["PhoneNumber → StateAvg","PhoneNumber → StateAvg","MeasureCode → ZIPCode","MeasureCode → State","PhoneNumber → City","ZIPCode → PhoneNumber","MeasureCode, State → Condition","MeasureCode  → City"]
contrainte_erronee = "MeasureCode → StateAvg"
contraintes_systemes = ["PhoneNumber → State, Condition"]


contraintes_testees = generer_contrainte(df, contrainte_erronee, contraintes_systemes)

print("les contraintes à tester:", contraintes_testees)

contraintes_valides = check_functional_dependencies(df, contraintes_testees)


print("les contraintes valides:")
for contrainte in contraintes_valides:
    print(contrainte)



les contraintes à tester: ['MeasureCode, MeasureName → StateAvg', 'MeasureCode, MeasureName, State → StateAvg', 'MeasureCode → StateAvg, MeasureName', 'MeasureCode → StateAvg, MeasureName, Condition', 'MeasureName → StateAvg', 'MeasureCode → StateAvg', 'MeasureCode → MeasureName']
les contraintes valides:
MeasureCode, MeasureName, State → StateAvg
MeasureCode → MeasureName


**Trouver la contrainte avec la structure la plus similaire à la contrainte erronée**

In [28]:

def jaccard_coefficient(constraint1, constraint2):
    set1 = set(constraint1)
    set2 = set(constraint2)
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union)

def trouverContraintePlusSimilaire(contrainteErronee, ensembleContraintes):
    # Initialisation
    maxJaccard = 0
    contraintePlusSimilaire = None



    # Parcourir les contraintes
    for contrainte in ensembleContraintes:

        # Calculer le coefficient de Jaccard
        coefficientJaccard = jaccard_coefficient(contrainteErronee, contrainte)

        # Mise à jour de la contrainte la plus similaire
        if coefficientJaccard > maxJaccard:
            maxJaccard = coefficientJaccard
            contraintePlusSimilaire = contrainte

    # Retourner la contrainte la plus similaire

    return contraintePlusSimilaire



In [29]:
# Exemple d'utilisation sans le test de validité qui a donner s'un seul resultat

contrainte_erronee = "State  → City"
contraintes = generer_contrainte(df, contrainte_erronee, contraintes_systemes)
contraintes_systemes = []
print("les contraintes générées sont :",contraintes )
resultat = trouverContraintePlusSimilaire(contrainte_erronee, contraintes)
print("Contrainte la plus similaire :", resultat)


les contraintes générées sont : ['State, ZIPCode → City', 'State, ZIPCode, PhoneNumber → City', 'State → City, ZIPCode', 'State → City, PhoneNumber', 'State → City, ZIPCode, PhoneNumber', 'ZIPCode → City', 'State → ZIPCode', 'State → PhoneNumber']
Contrainte la plus similaire : State, ZIPCode → City


test de similarité avec la longueur

In [30]:
def levenshtein_distance(str1, str2):
    m = len(str1)
    n = len(str2)

    dp = [[0] * (n + 1) for _ in range(m + 1)]

    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            cost = 0 if str1[i - 1] == str2[j - 1] else 1
            dp[i][j] = min(dp[i - 1][j] + 1, dp[i][j - 1] + 1, dp[i - 1][j - 1] + cost)

    return dp[m][n]

def most_similar_constraint(erroneous_constraint, constraint_set):
    min_distance = float('inf')
    most_similar = None

    for proposal in constraint_set:
        distance = levenshtein_distance(erroneous_constraint, proposal)
        if distance < min_distance:
            min_distance = distance
            most_similar = proposal

    return most_similar

# Exemple d'utilisation

contrainte_erronee = "PhoneNumber → State, Condition"
contraintes = generer_contrainte(df, contrainte_erronee, contraintes_systemes)
contraintes_systemes = []
print("les contraintes générées sont :",contraintes )
resultat = most_similar_constraint(contrainte_erronee, contraintes)
print("Contrainte la plus similaire :", resultat)


les contraintes générées sont : ['PhoneNumber, StateAvg → Condition, State', 'PhoneNumber, StateAvg, MeasureName → Condition, State', 'PhoneNumber, StateAvg, MeasureCode → Condition, State', 'PhoneNumber, MeasureName, MeasureCode → Condition, State', 'PhoneNumber → Condition, State, ProviderNumber', 'PhoneNumber → Condition, State, ProviderNumber, ZIPCode', 'PhoneNumber → State', 'StateAvg → Condition, State', 'PhoneNumber → State, ProviderNumber']
Contrainte la plus similaire : PhoneNumber → State


**Choix de la meilleur contrainte a prendre pour la reparation hybride**

In [31]:


def Meilleurs_contrainte_validité(df, constraints):
    valid_constraints = []
    max_valid_rows = 0

    for constraint in constraints:
        lhs, rhs = constraint.split(" → ")
        lhs = lhs.strip().split(', ')
        rhs = rhs.strip().split(', ')

        # Vérification si toutes les colonnes existent dans le DataFrame
        for col in lhs + rhs:
            if col not in df.columns:
                raise ValueError(f"La colonne '{col}' n'existe pas dans le DataFrame.")

        # Agréger les données en fonction de lhs et vérifier la dépendance fonctionnelle pour chaque combinaison unique
        grouped_data = df.groupby(lhs)[rhs].nunique()

        # Compter le nombre de lignes valides pour cette contrainte
        num_valid_rows = (grouped_data == 1).all(axis=1).sum()

        # Vérifier si le nombre de lignes valides est supérieur au maximum trouvé jusqu'à présent
        if num_valid_rows > max_valid_rows:
            max_valid_rows = num_valid_rows
            valid_constraints = [constraint]
        elif num_valid_rows == max_valid_rows:
            valid_constraints.append(constraint)

    return valid_constraints


**reparation des donnes**

In [32]:
def Reparer_donnes(df, contrainte_choisie, contraintes_systemes):
    # utilisée dans le cas de la reparation hybride
    pass


**supression de la contrainte erronéé et son remplacement dans la liste des contraintes dy systeme**

In [33]:
def remplacer_contrainte(contraintes_systeme, contrainte_erronee, contrainte_remplacante):
    if contrainte_erronee in contraintes_systeme:
        contraintes_systeme.remove(contrainte_erronee)
        contraintes_systeme.append(contrainte_remplacante)




**fonction pricipale pour la réparation**

In [34]:

def Reparer(df, liste_contraintes_erronees, contraintes_systemes):
    # Liste pour stocker les résultats finaux des contraintes réparées
    contraintes_reparees = []
    contraintes_reparees_hybride = []
    for contrainte_erronee in liste_contraintes_erronees:
        # Générer les contraintes potentielles à partir de la contrainte erronée et des contraintes système
        contraintes_generees = generer_contrainte(df, contrainte_erronee, contraintes_systemes)

        # Valider les contraintes générées pour déterminer lesquelles sont correctes et applicables
        contraintes_valides = check_functional_dependencies(df, contraintes_generees)

        # Trouver la contrainte valide la plus similaire à la contrainte erronée
        if len(contraintes_valides) > 0:

            if len(contraintes_valides) > 1:
                contrainte_finale = most_similar_constraint(contrainte_erronee, contraintes_valides)
            else:
                contrainte_finale = contraintes_valides[0]  # Prendre la première contrainte valide directement

            # Ajouter la contrainte réparée à la liste des contraintes réparées
            contraintes_reparees.append((contrainte_erronee, contrainte_finale))


        else:

            Contraintes_choisies= Meilleurs_contrainte_validité(df, contraintes_generees)

            if len(Contraintes_choisies) > 1:
                contrainte_choisie = most_similar_constraint(contrainte_erronee, Contraintes_choisies)
            else:
                contrainte_choisie = Contraintes_choisies[0]  # Prendre la première contrainte valide directement

            contraintes_reparees_hybride.append((contrainte_erronee, contrainte_choisie))

            Reparer_donnes(df,contrainte_choisie,contraintes_systemes)

    return contraintes_reparees,contraintes_reparees_hybride


**Test pour calculer les differentes mesures**

In [35]:
# Exemple d'utilisation avec une liste de contraintes erronées

#"PhoneNumber → State" de 1 à 6


contrainte_erronee = ["MeasureCode  → City","PhoneNumber → City, Condition", "MeasureCode → Condition, City", "MeasureCode → MeasureName, City", "PhoneNumber → ZIPCode, Condition", "MeasureCode, State → StateAvg, ZIPCode"]
contraintes_systemes_avant = ["MeasureCode  → City","PhoneNumber → City, Condition", "MeasureCode → Condition, City", "MeasureCode → MeasureName, City", "PhoneNumber → ZIPCode, Condition", "MeasureCode, State → StateAvg, ZIPCode","ZIPCode, State → StateAvg","PhoneNumber → MeasureName","MeasureCode → StateAvg","MeasureCode, ZIPCode  → State","PhoneNumber, StateAvg  → State"]
contraintes_reparees, contraintes_choisies = Reparer(df, contrainte_erronee, contraintes_systemes_avant)

for contrainte_erronee, contrainte_reparee in contraintes_reparees:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation: {contrainte_reparee}")
for contrainte_erronee, contrainte_choisie in contraintes_choisies:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation hybride: {contrainte_choisie}")


Contrainte erronée : MeasureCode  → City => sa réparation: MeasureCode → MeasureName
Contrainte erronée : PhoneNumber → City, Condition => sa réparation: PhoneNumber → City
Contrainte erronée : MeasureCode → Condition, City => sa réparation: MeasureCode → Condition
Contrainte erronée : MeasureCode → MeasureName, City => sa réparation: MeasureCode → MeasureName
Contrainte erronée : PhoneNumber → ZIPCode, Condition => sa réparation: PhoneNumber → ZIPCode
Contrainte erronée : MeasureCode, State → StateAvg, ZIPCode => sa réparation: MeasureCode, State → StateAvg


In [36]:
# Exemple d'utilisation avec une liste de contraintes erronées

#"PhoneNumber → State"  7


contrainte_erronee = ["City, State → StateAvg", ]
contraintes_systemes_avant = [
"ZIPCode → State",
"PhoneNumber → ZIPCode",
"MeasureCode → MeasureName",
"MeasureCode → Condition",
"PhoneNumber → City",
"PhoneNumber → State","MeasureCode → StateAvg","State → StateAvg"]
contraintes_reparees, contraintes_choisies = Reparer(df, contrainte_erronee, contraintes_systemes_avant)

for contrainte_erronee, contrainte_reparee in contraintes_reparees:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation: {contrainte_reparee}")
for contrainte_erronee, contrainte_choisie in contraintes_choisies:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation hybride: {contrainte_choisie}")

Contrainte erronée : City, State → StateAvg => sa réparation: City, State, MeasureName → StateAvg


In [37]:
# Exemple d'utilisation avec une liste de contraintes erronées

# 8


contrainte_erronee = ["PhoneNumber → MeasureName"]
contraintes_systemes_avant = [
"ZIPCode → State"]
contraintes_reparees, contraintes_choisies = Reparer(df, contrainte_erronee, contraintes_systemes_avant)

for contrainte_erronee, contrainte_reparee in contraintes_reparees:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation: {contrainte_reparee}")
for contrainte_erronee, contrainte_choisie in contraintes_choisies:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation hybride: {contrainte_choisie}")

Contrainte erronée : PhoneNumber → MeasureName => sa réparation: MeasureCode → MeasureName


In [38]:
# Exemple d'utilisation avec une liste de contraintes erronées

#"PhoneNumber → State"
#ZIPCode → State  9 et 10

contrainte_erronee = ["ZIPCode → State, Condition","PhoneNumber, StateAvg  → State"]
contraintes_systemes_avant = ["StateAvg → State","PhoneNumber, StateAvg  → State"]
contraintes_reparees, contraintes_choisies = Reparer(df, contrainte_erronee, contraintes_systemes_avant)

for contrainte_erronee, contrainte_reparee in contraintes_reparees:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation: {contrainte_reparee}")
for contrainte_erronee, contrainte_choisie in contraintes_choisies:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation hybride: {contrainte_choisie}")

Contrainte erronée : ZIPCode → State, Condition => sa réparation: ZIPCode → State
Contrainte erronée : PhoneNumber, StateAvg  → State => sa réparation: PhoneNumber → State


In [39]:
# Exemple d'utilisation avec une liste de contraintes erronées

#State, MeasureName → StateAvg  11


contrainte_erronee = ["State → StateAvg"]
contraintes_systemes_avant = [
"ZIPCode → State",
"PhoneNumber → ZIPCode",
"MeasureCode → MeasureName",
"MeasureCode → Condition",
"PhoneNumber → City",
"PhoneNumber → State","MeasureCode → StateAvg","State → StateAvg"]
contraintes_reparees, contraintes_choisies = Reparer(df, contrainte_erronee, contraintes_systemes_avant)

for contrainte_erronee, contrainte_reparee in contraintes_reparees:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation: {contrainte_reparee}")
for contrainte_erronee, contrainte_choisie in contraintes_choisies:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation hybride: {contrainte_choisie}")

Contrainte erronée : State → StateAvg => sa réparation: State, MeasureName → StateAvg


In [46]:
# Exemple d'utilisation avec une liste de contraintes erronées

#State, MeasureName → StateAvg  11


contrainte_erronee = ["MeasureCode → StateAvg"]
contraintes_systemes_avant = [
"ZIPCode → State",
"PhoneNumber → ZIPCode",
"MeasureCode → MeasureName",
"MeasureCode → Condition",
"PhoneNumber → City",
"PhoneNumber → State","MeasureCode → StateAvg","State → StateAvg"]
contraintes_reparees, contraintes_choisies = Reparer(df, contrainte_erronee, contraintes_systemes_avant)

for contrainte_erronee, contrainte_reparee in contraintes_reparees:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation: {contrainte_reparee}")
for contrainte_erronee, contrainte_choisie in contraintes_choisies:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation hybride: {contrainte_choisie}")

Contrainte erronée : MeasureCode → StateAvg => sa réparation: MeasureCode, MeasureName, State → StateAvg







# **2.   Evaluation**





**Evaluation du temps d'exécution**

Test avec une seule fd erronée

In [40]:


def evaluer_temps_execution(df, liste_contraintes_erronees, contraintes_systemes):
    debut_execution = time.time()
    contraintes_reparees, contraintes_reparees_hybride = Reparer(df, liste_contraintes_erronees, contraintes_systemes)
    fin_execution = time.time()

    temps_execution = fin_execution - debut_execution
    return temps_execution, contraintes_reparees, contraintes_reparees_hybride







contrainte_erronee = ["State → StateAvg"]
contraintes_systemes_avant = [
"ZIPCode → State",
"PhoneNumber → ZIPCode",
"MeasureCode → MeasureName",
"MeasureCode → Condition",
"PhoneNumber → City",
"PhoneNumber → State","MeasureCode → StateAvg","State → StateAvg"]
# Évaluer le temps d'exécution et obtenir les résultats de réparation
temps_execution, contraintes_reparees, contraintes_choisies = evaluer_temps_execution(df, contrainte_erronee, contraintes_systemes_avant)

for contrainte_erronee, contrainte_reparee in contraintes_reparees:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation: {contrainte_reparee}")
for contrainte_erronee, contrainte_choisie in contraintes_choisies:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation hybride: {contrainte_choisie}")
# Afficher le temps d'exécution
print(f"Temps d'exécution total : {temps_execution} secondes")


Contrainte erronée : State → StateAvg => sa réparation: State, MeasureName → StateAvg
Temps d'exécution total : 0.4274735450744629 secondes


Test avec 4 DFs erronées

In [41]:


def evaluer_temps_execution(df, liste_contraintes_erronees, contraintes_systemes):
    debut_execution = time.time()
    contraintes_reparees, contraintes_reparees_hybride = Reparer(df, liste_contraintes_erronees, contraintes_systemes)
    fin_execution = time.time()

    temps_execution = fin_execution - debut_execution
    return temps_execution, contraintes_reparees, contraintes_reparees_hybride







contrainte_erronee = ["MeasureCode  → City","PhoneNumber → City, Condition", "MeasureCode → Condition, City", "MeasureCode → MeasureName, City"]
contraintes_systemes_avant = ["MeasureCode  → City","PhoneNumber → City, Condition", "MeasureCode → Condition, City", "MeasureCode → MeasureName, City"]
# Évaluer le temps d'exécution et obtenir les résultats de réparation
temps_execution, contraintes_reparees, contraintes_choisies = evaluer_temps_execution(df, contrainte_erronee, contraintes_systemes_avant)

for contrainte_erronee, contrainte_reparee in contraintes_reparees:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation: {contrainte_reparee}")
for contrainte_erronee, contrainte_choisie in contraintes_choisies:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation hybride: {contrainte_choisie}")
# Afficher le temps d'exécution
print(f"Temps d'exécution total : {temps_execution} secondes")


Contrainte erronée : MeasureCode  → City => sa réparation: MeasureCode → MeasureName
Contrainte erronée : PhoneNumber → City, Condition => sa réparation: PhoneNumber → City
Contrainte erronée : MeasureCode → Condition, City => sa réparation: MeasureCode → Condition
Contrainte erronée : MeasureCode → MeasureName, City => sa réparation: MeasureCode → MeasureName
Temps d'exécution total : 2.2148842811584473 secondes


Test avec 8 DFs erronées

In [42]:


def evaluer_temps_execution(df, liste_contraintes_erronees, contraintes_systemes):
    debut_execution = time.time()
    contraintes_reparees, contraintes_reparees_hybride = Reparer(df, liste_contraintes_erronees, contraintes_systemes)
    fin_execution = time.time()

    temps_execution = fin_execution - debut_execution
    return temps_execution, contraintes_reparees, contraintes_reparees_hybride






contrainte_erronee = ["MeasureCode  → City","PhoneNumber → City, Condition", "MeasureCode → Condition, City", "MeasureCode → MeasureName, City", "PhoneNumber → ZIPCode, Condition", "MeasureCode, State → StateAvg, ZIPCode","ZIPCode → State, Condition","PhoneNumber, StateAvg  → State"]
contraintes_systemes_avant = ["MeasureCode  → City","PhoneNumber → City, Condition", "MeasureCode → Condition, City", "MeasureCode → MeasureName, City", "PhoneNumber → ZIPCode, Condition", "MeasureCode, State → StateAvg, ZIPCode","ZIPCode, State → StateAvg","PhoneNumber → MeasureName","MeasureCode → StateAvg","MeasureCode, ZIPCode  → State","PhoneNumber, StateAvg  → State"]
# Évaluer le temps d'exécution et obtenir les résultats de réparation
temps_execution, contraintes_reparees, contraintes_choisies = evaluer_temps_execution(df, contrainte_erronee, contraintes_systemes_avant)

for contrainte_erronee, contrainte_reparee in contraintes_reparees:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation: {contrainte_reparee}")
for contrainte_erronee, contrainte_choisie in contraintes_choisies:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation hybride: {contrainte_choisie}")
# Afficher le temps d'exécution
print(f"Temps d'exécution total : {temps_execution} secondes")


Contrainte erronée : MeasureCode  → City => sa réparation: MeasureCode → MeasureName
Contrainte erronée : PhoneNumber → City, Condition => sa réparation: PhoneNumber → City
Contrainte erronée : MeasureCode → Condition, City => sa réparation: MeasureCode → Condition
Contrainte erronée : MeasureCode → MeasureName, City => sa réparation: MeasureCode → MeasureName
Contrainte erronée : PhoneNumber → ZIPCode, Condition => sa réparation: PhoneNumber → ZIPCode
Contrainte erronée : MeasureCode, State → StateAvg, ZIPCode => sa réparation: MeasureCode, State → StateAvg
Contrainte erronée : ZIPCode → State, Condition => sa réparation: ZIPCode → State
Contrainte erronée : PhoneNumber, StateAvg  → State => sa réparation: PhoneNumber → State
Temps d'exécution total : 3.2804675102233887 secondes


**Evaluation de la variation entre les exécutions**

In [43]:
def execute_iterations(num_iterations):
    # Import the required libraries and functions here

    for iteration in range(num_iterations):
        print(f"Iteration {iteration + 1}")

        # Exemple d'utilisation avec une liste de contraintes erronées
        contrainte_erronee = ["City → State","State → StateAvg","MeasureCode → Condition, PhoneNumber","MeasureCode, State → MeasureName","ZIPCode → City","PhoneNumber → StateAvg"]
        contraintes_systemes_avant = ["City → State","State → StateAvg","MeasureCode → Condition, PhoneNumber","MeasureCode, State → MeasureName","ZIPCode → City","PhoneNumber → StateAvg","PhoneNumber → ZIPCode"]

        contraintes_reparees, contraintes_choisies, contraintes_systemes_après = Reparer(df, contrainte_erronee, contraintes_systemes_avant)

        # Afficher les résultats de réparation
        print("Résultats de réparation:")
        for contrainte_erronee, contrainte_reparee in contraintes_reparees:
            print(f"Contrainte erronée : {contrainte_erronee} => sa réparation: {contrainte_reparee}")
        for contrainte_erronee, contrainte_choisie in contraintes_choisies:
            print(f"Contrainte erronée : {contrainte_erronee} => sa réparation hybride: {contrainte_choisie}")
        print("Après cette réparation, les contraintes du notre système sont: ", contraintes_systemes_après)
        print("")

if __name__ == "__main__":
    num_iterations = 5  # You can adjust the number of iterations here
    execute_iterations(num_iterations)


Iteration 1


ValueError: ignored

**Analyse de la consommation de ressources**

test sur 6 FDs erronées

In [ ]:

# Exemple d'utilisation avec une liste de contraintes erronées
contrainte_erronee = ["City → State","State → StateAvg","MeasureCode → Condition, PhoneNumber","MeasureCode, State → MeasureName","ZIPCode → City","PhoneNumber → StateAvg"]
contraintes_systemes_avant = ["City → State","State → StateAvg","MeasureCode → Condition, PhoneNumber","MeasureCode, State → MeasureName","ZIPCode → City","PhoneNumber → StateAvg","PhoneNumber → ZIPCode"]
memory_before_repair = psutil.virtual_memory().used / (1024 ** 2)  # Convertir en Mo
print(f"Mémoire utilisée avant la réparation: {memory_before_repair:.2f} Mo")
contraintes_reparees, contraintes_choisies, contraintes_systemes_aprés = Reparer(df, contrainte_erronee, contraintes_systemes_avant)

memory_after_repair = psutil.virtual_memory().used / (1024 ** 2)  # Convertir en Mo
memory_total = memory_after_repair-memory_before_repair
# Afficher les résultats de réparation
for contrainte_erronee, contrainte_reparee in contraintes_reparees:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation: {contrainte_reparee}")
for contrainte_erronee, contrainte_choisie in contraintes_choisies:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation hybride: {contrainte_choisie}")
print("Après cette réparation, les contraintes du notre système sont: ", contraintes_systemes_après)

print(f"Mémoire utilisée après la réparation: {memory_after_repair:.2f} Mo")

print(f"Mémoire totale  utilisée :  {memory_total:.2f} Mo")





test sur une seule FD erronée

In [ ]:


# Exemple d'utilisation avec une liste de contraintes erronées
contrainte_erronee = ["City → State"]
contraintes_systemes_avant = ["City → State","State → StateAvg","MeasureCode → Condition, PhoneNumber","MeasureCode, State → MeasureName","ZIPCode → City","PhoneNumber → StateAvg","PhoneNumber → ZIPCode"]
memory_before_repair = psutil.virtual_memory().used / (1024 ** 2)  # Convertir en Mo
print(f"Mémoire utilisée avant la réparation: {memory_before_repair:.2f} Mo")
contraintes_reparees, contraintes_choisies, contraintes_systemes_aprés = Reparer(df, contrainte_erronee, contraintes_systemes_avant)

memory_after_repair = psutil.virtual_memory().used / (1024 ** 2)  # Convertir en Mo
memory_total = memory_after_repair-memory_before_repair
# Afficher les résultats de réparation
for contrainte_erronee, contrainte_reparee in contraintes_reparees:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation: {contrainte_reparee}")
for contrainte_erronee, contrainte_choisie in contraintes_choisies:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation hybride: {contrainte_choisie}")
print("Après cette réparation, les contraintes du notre système sont: ", contraintes_systemes_après)

print(f"Mémoire utilisée après la réparation: {memory_after_repair:.2f} Mo")

print(f"Mémoire totale  utilisée :  {memory_total:.2f} Mo")







**Comparison des contraintes réparées avec les contraintes correctes d'origine**

**Test avec des erreurs en supprimant des attribits de la partie gauche de la contrainte d'origine**

In [ ]:
contrainte_erronee = ["MeasureCode → StateAvg"]
contraintes_systemes_avant = ["MeasureCode → StateAvg"]
contraintes_reparees, contraintes_choisies, contraintes_systemes_aprés = Reparer(df, contrainte_erronee, contraintes_systemes_avant)

for contrainte_erronee, contrainte_reparee in contraintes_reparees:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation: {contrainte_reparee}")
for contrainte_erronee, contrainte_choisie in contraintes_choisies:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation hybride: {contrainte_choisie}")
print("Aprés cette reparation les contraintes du notre systeme sont: ", contraintes_systemes_aprés)




**Test avec des erreurs en remplaçant un des attribits de la partie gauche de la contrainte d'origine par d'autres**

In [ ]:
contrainte_erronee = ["State, City → StateAvg","ZIPCode → Condition","City → MeasureName","State → ZIPCode","MeasureName → State","ZIPCode, State → StateAvg"]
contraintes_systemes_avant = ["MeasureCode → StateAvg"]
contraintes_reparees, contraintes_choisies, contraintes_systemes_aprés = Reparer(df, contrainte_erronee, contraintes_systemes_avant)

for contrainte_erronee, contrainte_reparee in contraintes_reparees:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation: {contrainte_reparee}")
for contrainte_erronee, contrainte_choisie in contraintes_choisies:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation hybride: {contrainte_choisie}")
print("Aprés cette reparation les contraintes du notre systeme sont: ", contraintes_systemes_aprés)



**Test avec des erreurs en remplaçant un des attribits de la partie droite de la contrainte d'origine par d'autres**

In [ ]:
contrainte_erronee = ["PhoneNumber → StateAvg","PhoneNumber → StateAvg","MeasureCode → ZIPCode","MeasureCode → State","PhoneNumber → City","ZIPCode → PhoneNumber","MeasureCode, State → Condition","MeasureCode  → City"]
contraintes_systemes_avant = ["MeasureCode → StateAvg"]
contraintes_reparees, contraintes_choisies, contraintes_systemes_aprés = Reparer(df, contrainte_erronee, contraintes_systemes_avant)

for contrainte_erronee, contrainte_reparee in contraintes_reparees:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation: {contrainte_reparee}")
for contrainte_erronee, contrainte_choisie in contraintes_choisies:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation hybride: {contrainte_choisie}")
print("Aprés cette reparation les contraintes du notre systeme sont: ", contraintes_systemes_aprés)



**Test avec des erreurs en ajoutant des attribits en plus à la partie gauche de la contrainte d'origine**

In [ ]:
contrainte_erronee = ["MeasureCode, State, Condition → StateAvg","ZIPCode, PhoneNumber → State","PhoneNumber, Condition → ZIPCode","MeasureCode, State → MeasureName","MeasureCode, State → Condition","PhoneNumber, StateAvg → City","PhoneNumber, Condition → State"]
contraintes_systemes_avant = []
contraintes_reparees, contraintes_choisies, contraintes_systemes_aprés = Reparer(df, contrainte_erronee, contraintes_systemes_avant)

for contrainte_erronee, contrainte_reparee in contraintes_reparees:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation: {contrainte_reparee}")
for contrainte_erronee, contrainte_choisie in contraintes_choisies:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation hybride: {contrainte_choisie}")
print("Aprés cette reparation les contraintes du notre systeme sont: ", contraintes_systemes_aprés)


**Test avec des erreurs en ajoutant des attribits en plus à la partie droite de la contrainte d'origine**

In [ ]:
contrainte_erronee = ["PhoneNumber → State, StateAvg","PhoneNumber → City, Condition","MeasureCode → Condition, City","MeasureCode → MeasureName, City","PhoneNumber → ZIPCode, Condition","ZIPCode → State, PhoneNumber","MeasureCode, State → StateAvg, ZIPCode"]
contraintes_systemes_avant = []
contraintes_reparees, contraintes_choisies, contraintes_systemes_aprés = Reparer(df, contrainte_erronee, contraintes_systemes_avant)

for contrainte_erronee, contrainte_reparee in contraintes_reparees:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation: {contrainte_reparee}")
for contrainte_erronee, contrainte_choisie in contraintes_choisies:
    print(f"Contrainte erronée : {contrainte_erronee} => sa réparation hybride: {contrainte_choisie}")
print("Aprés cette reparation les contraintes du notre systeme sont: ", contraintes_systemes_aprés)

